In [17]:
!pip install altair vl-convert-python

  Using cached vl_convert_python-1.7.0-cp37-abi3-macosx_10_12_x86_64.whl.metadata (5.2 kB)
Using cached vl_convert_python-1.7.0-cp37-abi3-macosx_10_12_x86_64.whl (28.2 MB)

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [4]:
import altair as alt
import pandas as pd
import numpy as np
import json
import os, re

In [100]:
def parse_arch(arch):
    # Pattern: "<C> Columns, <P> PLCUs, <G> PLCGs, <B> bits
    arch_pattern = r"^(\d+) Columns, (\d+) PLCUs, (\d+) PLCGs, (\d+) bits$"
    match = re.match(arch_pattern, arch)
    vals = {
        "n_cols": int(match.group(1)),
        "n_plcus": int(match.group(2)),
        "n_plcgs": int(match.group(3)),
        "n_bits": int(match.group(4)),
    }
    return nice_arch_name(vals)

def nice_arch_name(parsed_arch, with_sensitivity=True):
    if parsed_arch.get("noiseless"):
        return f"Noiseless {parsed_arch['n_bits']} bits"
    sensitivity = ""
    if parsed_arch.get("sensitivity") and with_sensitivity:
        sensitivity = f", High"
    elif with_sensitivity:
        sensitivity = f", Low"
    return f"{parsed_arch['n_cols']} Columns, {parsed_arch['n_plcus']} PLCUs, {parsed_arch['n_bits']} bits{sensitivity}"


def filename_to_arch_name(filename, with_sensitivity=True):
    # Filename pattern 1: <someprefix>_n_columns_<C>_n_plcus_<P>_n_bits_<B>_<somesuffix>.json
    # Filename pattern 2: <someprefix>_n_bits_<B>_no_noise.json
    if "original" in filename:
        return "Original FP32"
    if "_no_noise" in filename:
        pattern = r"^.*_n_bits_(\d+)_no_noise\.json$"
        match = re.match(pattern, filename)
        return nice_arch_name({
            "n_bits": int(match.group(1)),
            "noiseless": True,
        }, with_sensitivity=with_sensitivity)
    sensitive = "_sensitive" in filename
    pattern = r"^.*_n_columns_(\d+)_n_plcus_(\d+)_n_bits_(\d+).*?\.json$"
    match = re.match(pattern, filename)
    return nice_arch_name({
        "n_cols": int(match.group(1)),
        "n_plcus": int(match.group(2)),
        "n_bits": int(match.group(3)),
        "sensitivity": sensitive,
        "noiseless": False,
    }, with_sensitivity=with_sensitivity)




def read_energy_results():
    filename = "FinalResults/architecture_exploration.json"
    with open(filename, "r") as f:
        raw_data = json.load(f)
    # Total energies.
    raw_total_energies = raw_data["per_architecture"]
    total_energies = {
        "name": [],
        "architecture": [],
        "energy": [],
        "ordering": [],
    }
    for name, component_energies in raw_total_energies.items():
        for arch, energy in component_energies:
            total_energies["name"].append(name)
            total_energies["architecture"].append(parse_arch(arch))
            total_energies["energy"].append(energy*1e12) # Convert to pJ
            total_energies["ordering"].append(len(total_energies['ordering']))
    total_energies = pd.DataFrame(total_energies)
    # Effciencies.
    raw_efficiencies = raw_data["Energy Efficiency"][""]
    efficiencies = {
        "architecture": [],
        "efficiency": [],
        "ordering": [],
    }
    for arch, efficiency in raw_efficiencies:
        efficiencies["architecture"].append(parse_arch(arch))
        efficiencies["efficiency"].append(efficiency)
        efficiencies["ordering"].append(len(efficiencies['ordering']))
    efficiencies = pd.DataFrame(efficiencies)
    # Densities.
    raw_densities = raw_data["Compute Density"][""]
    densities = {
        "architecture": [],
        "density": [],
        "ordering": [],
    }
    for arch, density in raw_densities:
        densities["architecture"].append(parse_arch(arch))
        densities["density"].append(density)
        densities["ordering"].append(len(densities['ordering']))
    densities = pd.DataFrame(densities)
    # Done.
    return total_energies, efficiencies, densities
    

total_energies, efficiencies, densities = read_energy_results()




def read_inference_results(file_prefix: str = "isolated_model_results_"):
    # Get all filenames starting with the prefix and ending with ".json"
    filenames = [f for f in os.listdir("FinalResults") if f.startswith(file_prefix) and f.endswith(".json")]
    inference_results = {
        "architecture": [],
        "full_architecture": [],
        "accuracy": [],
        "cross_entropy_loss": [],
        "ordering": [],
        "architecture_type": [],
    }
    for filename in filenames:
        with open(os.path.join("FinalResults", filename), "r") as f:
            raw_data = json.load(f)
        architecture = filename_to_arch_name(filename, with_sensitivity=False)
        full_architecture = filename_to_arch_name(filename, with_sensitivity=True)
        accuracy = raw_data["final_acc"]
        cross_entropy_loss = raw_data["final_loss"]
        inference_results["architecture"].append(architecture)
        inference_results["full_architecture"].append(full_architecture)
        inference_results["accuracy"].append(accuracy)
        inference_results["cross_entropy_loss"].append(cross_entropy_loss)
        # Sort order allows nice display of the architectures in the plot.
        if "original" in filename:
            ordering = f"000_{architecture}"
            arch_type = "Noiseless"
        elif "_no_noise" in filename:
            ordering = f"001_{architecture}"
            arch_type = "Noiseless"
        elif "_sensitive" in filename:
            ordering = f"sensitive {architecture}"
            arch_type = "High Sensitivity"
        else:
            ordering = f"{architecture}"
            arch_type = "Low Sensitivity"
        inference_results["architecture_type"].append(arch_type)
        inference_results["ordering"].append(ordering)
    # Sort the results by the ordering.
    inference_results = pd.DataFrame(inference_results)
    inference_results = inference_results.sort_values(by="ordering").reset_index(drop=True)
    inference_results['ordering'] = np.arange(len(inference_results))
    return inference_results


inference_results = read_inference_results()
inference_results.to_csv("FinalResults/inference_results.csv", index=False)

In [ ]:
# Make the following plots:
# 1. Energy vs Architecture
# 2. Efficiency vs Architecture
# 3. Density vs Architecture
# 4. Accuracy vs Architecture (include with sensitivity)
# 5. Cross Entropy Loss vs Architecture (include with sensitivity)

bar_width = 0.5
tick_font_size = 14
title_font_size = 18
legend_font_size = 14
legend_title_font_size = 16


def plot_total_energies(total_energies):
    # Plot energy vs architecture
    chart = alt.Chart(total_energies).mark_bar(width={'band' : bar_width}).encode(
        x=alt.X("architecture:N", title="Architecture",),
        y=alt.Y("energy:Q", title="Energy (pJ/MAC)",),
        color=alt.Color("name:N", title="Component"),
        tooltip=["name:N", "energy:Q"]
    ).properties(
        width=500,
        height=300,
    ).configure_legend(
        labelFontSize=legend_font_size,
        titleFontSize=legend_title_font_size,
    ).configure_axis(
        labelFontSize=tick_font_size,
        titleFontSize=title_font_size,
    ).configure_axisX(
        labelAngle=-45,
    )
    chart.save("FinalResults/total_energies.png")
    return chart

def plot_efficiencies(efficiencies):
    # Plot efficiency vs architecture
    chart = alt.Chart(efficiencies).mark_bar().encode(
        x=alt.X("architecture:N", title="Architecture"),
        y=alt.Y("efficiency:Q", title="Efficiency (TOPS/Watt)"),
        tooltip=["architecture:N", "efficiency:Q"]
    ).properties(
        width=500,
        height=300,
    ).configure_legend(
        labelFontSize=legend_font_size,
        titleFontSize=legend_title_font_size,
    ).configure_axis(
        labelFontSize=tick_font_size,
        titleFontSize=title_font_size,
    ).configure_axisX(
        labelAngle=-45,
    )
    chart.save("FinalResults/energie_efficiencies.png")
    return chart


def plot_densities(densities):
    # Plot density vs architecture
    chart = alt.Chart(densities).mark_bar().encode(
        x=alt.X("architecture:N", title="Architecture"),
        y=alt.Y("density:Q", title="Density (TOPS/mm^2)"),
        tooltip=["architecture:N", "density:Q"]
    ).properties(
        width=500,
        height=300,
    ).configure_legend(
        labelFontSize=legend_font_size,
        titleFontSize=legend_title_font_size,
    ).configure_axis(
        labelFontSize=tick_font_size,
        titleFontSize=title_font_size,
    ).configure_axisX(
        labelAngle=-45,
    )
    chart.save("FinalResults/compute_densities.png")
    return chart


def plot_accuracies(inference_results, with_sensitivity=True):
    # Plot accuracy vs architecture
    chart = alt.Chart(inference_results).mark_bar(clip=True).encode(
        x=alt.X(
            "full_architecture:N", title="Architecture",
            sort=alt.SortField(field="ordering", order="ascending"),
        ),
        y=alt.Y("accuracy:Q", title="Accuracy", scale=alt.Scale(domain=[50, 80])),
        # color=alt.Color("architecture_type:N", title="Architecture Type"),
        # tooltip=["full_architecture:N", "accuracy:Q"]
    ).properties(
        width=800,
        height=600,
    ).configure_legend(
        labelFontSize=legend_font_size,
        titleFontSize=legend_title_font_size,
    ).configure_axis(
        labelFontSize=tick_font_size,
        titleFontSize=title_font_size,
        labelLimit=0,
    ).configure_axisX(
        labelAngle=-45,
    )
    chart.save("FinalResults/accuracies.png")
    return chart


def plot_cross_entropy_losses(inference_results):
    # Plot cross entropy loss vs architecture
    chart = alt.Chart(inference_results).mark_bar().encode(
        x=alt.X("architecture:N", title="Architecture", sort=alt.SortField(field="ordering", order="ascending")),
        y=alt.Y("cross_entropy_loss:Q", title="Cross Entropy Loss"),
        tooltip=["full_architecture:N", "cross_entropy_loss:Q"]
    ).properties(
        width=800,
        height=400,
    ).configure_legend(
        labelFontSize=legend_font_size,
        titleFontSize=legend_title_font_size,
    ).configure_axis(
        labelFontSize=tick_font_size,
        titleFontSize=title_font_size,
        labelLimit=0,
    ).configure_axisX(
        labelAngle=-45,
    )
    chart.save("FinalResults/cross_entropy_losses.png")
    return chart


# plot_total_energies(total_energies).display()
# plot_efficiencies(efficiencies).display()
# plot_densities(densities).display()
plot_accuracies(inference_results).display()
plot_cross_entropy_losses(inference_results).display()

alt.Chart(...)

alt.Chart(...)